In [ ]:
# IMPORTANT
import shapely.geometry as geom
import time

import seaborn as sns

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
#import plotly.express as px
from itertools import product
import matplotlib.pyplot as plt

import sys

class UpdatedLine:
    def __init__(self, p, eps_alpha, eps_beta, eps_v, eps_p, y_given_x_lambda, y_given_x_lambda_args={}):
        # store parameters
        self.p = p
        self.eps_alpha = eps_alpha
        self.eps_beta = eps_beta
        self.eps_v = eps_v
        self.eps_p = eps_p
        self.y_given_x_lambda = y_given_x_lambda
        self.y_given_x_lambda_args = y_given_x_lambda_args

    def build_curve(self, granularity=1000):
        x_vals = np.linspace(0.0, 1.0, num=granularity)
        # y_vals = self.y_given_x_lambda(x_vals, **self.y_given_x_lambda_args)
        y_vals = self.compute_y_vals_given_x_vals(x_vals)
        curve = np.array((x_vals,y_vals)).T
        self.curve = geom.LineString(curve)
      
    def compute_y_given_x(self, x_val):
        v = x_val  # x-axis is PPV (v)
        beta = (self.eps_p * (v**2 * (self.eps_alpha * (self.p - 1) - 1) + v * self.eps_v * (self.eps_alpha * (self.p - 1) - 1) + self.p * self.eps_v + v) + (self.p - 1) * (self.eps_alpha * (self.p - 1) * v * (v + self.eps_v) + self.p * self.eps_v) - self.eps_beta * (self.p - 1) * v * (self.p + self.eps_p) * (v + self.eps_v - 1)) / (self.eps_p * (self.p * self.eps_v - v**2 - v * self.eps_v + v) + (self.p - 1) * self.p * self.eps_v)
        return(beta)

    def compute_y_vals_given_x_vals(self, x_vals):
        v = np.array(x_vals)  # x-axis is PPV (v)
        beta = (self.eps_p * (v**2 * (self.eps_alpha * (self.p - 1) - 1) + v * self.eps_v * (self.eps_alpha * (self.p - 1) - 1) + self.p * self.eps_v + v) + (self.p - 1) * (self.eps_alpha * (self.p - 1) * v * (v + self.eps_v) + self.p * self.eps_v) - self.eps_beta * (self.p - 1) * v * (self.p + self.eps_p) * (v + self.eps_v - 1)) / (self.eps_p * (self.p * self.eps_v - v**2 - v * self.eps_v + v) + (self.p - 1) * self.p * self.eps_v)
        return(list(beta))

    def is_close_to_me(self, detector, tolerance, verbose=False):
        dist = detector.point.distance(self.curve)
        if verbose:
          print(f'X: {detector.point.x}, Y: {detector.point.y}')    
          print(f'Dist: {dist}')
          print()
        return abs(dist) < tolerance

    def plot(self, x_vals, axis=None): 
        if axis is None:
            plt.style.use('seaborn-whitegrid')
            fig = plt.figure(figsize=(8, 6), dpi=150)
            axis = plt.axes()
            axis.set(
                xlim=(0, 1), ylim=(0, 1), xlabel='PPV', ylabel='FPR'
            )
        axis.plot(*self.curve.xy)
        if axis is None:
            plt.show()

# A detector is a glorified point that remembers
class Detector():
  def __init__(self, x_val, y_val, verbose=False):
    self.x_val = x_val
    self.y_val = y_val
    self.point = geom.Point(x_val, y_val)
    self.satisfied = False
  
  def _my_area(g=None, r=None):
    # Might be useful?
    if g is not None:
      r = (1/2) * (1/(g-1))
    return np.pi * r ** 2

def parallel_is_in_feasible_region(lines, detector, tolerance, verbose=False):
      # Shortcut if the detector has already been satisfied
      # Wait, lol, this doesn't happen
      if detector.satisfied:
          return True

      for line in lines:
          if line.is_close_to_me(detector, tolerance, verbose):
              # point_on_line = line.curve.interpolate(line.curve.project(detector.point))
              # self.closest_segments.append(([detector.point.x, point_on_line.x], [detector.point.y, point_on_line.y]))
              detector.satisfied = True
              return True
      return False

def process(lines, detector, tolerance, verbose):
      return {
          'x': detector.point.x,
          'y': detector.point.y,
          'in_feasible_region': parallel_is_in_feasible_region(lines, detector, tolerance, verbose)
      }

# This update expects UpdatedLine classes in lines
class DetectorLattice:
    def __init__(self,
                 base_rate_1, 
                 base_rate_2, 
                 epsilon_range,
                 ppv_window=[0.0, 1.0],
                 verbose=False):
        self.ppv_window = ppv_window
        p = base_rate_2
        eps_p = base_rate_1 - base_rate_2
        epsilon_ranges = [epsilon_range] * 3
        # create a line for each set of epsilon values and store
        self.lines = [] 
        for eps_alpha, eps_beta, eps_v in tqdm(product(*epsilon_ranges), total=len(epsilon_range)**3, desc='Creating set of lines', disable=not verbose):
            line = UpdatedLine(p=p, eps_alpha=eps_alpha, eps_beta=eps_beta, eps_v=eps_v, eps_p=eps_p, y_given_x_lambda=None)
            line.build_curve()
            # line = PPVLine(p=p, eps_alpha=eps_alpha, eps_beta=eps_beta, eps_v=eps_v, eps_p=eps_p)
            self.lines.append(line)
        
        # DEBUG: Number of lines
        #print(len(self.lines))
        
        # self.lines = lines
        # for line in self.lines:
        #     line.build_curve()
      
        self.closest_segments = []
        self.detectors = []
    
    def is_in_feasible_region(self, detector, tolerance, verbose=False):
        # Shortcut if the detector has already been satisfied
        # Wait, lol, this doesn't happen
        if detector.satisfied:
            return True

        for line in self.lines:
            if line.is_close_to_me(detector, tolerance, verbose):
                # point_on_line = line.curve.interpolate(line.curve.project(detector.point))
                # self.closest_segments.append(([detector.point.x, point_on_line.x], [detector.point.y, point_on_line.y]))
                detector.satisfied = True
                return True
        return False

    def compute_area(self, num_grid_points=5, tolerance=(1/4) / 2, verbose=False, num_jobs=12):
        # define grid of points to sample
        x_points = np.linspace(0, 1, num_grid_points)
        y_points = np.linspace(0, 1, num_grid_points)
        for (x_val, y_val) in product(x_points, y_points):
            if self.ppv_window[0] < x_val <= self.ppv_window[1]:
                self.detectors.append(Detector(x_val, y_val))

        results_list = []
        for detector in tqdm(self.detectors, total=len(self.detectors), desc='Sampling detectors', disable=not verbose):
            results_list.append({
                'x': detector.point.x,
                'y': detector.point.y,
                'in_feasible_region': self.is_in_feasible_region(detector, tolerance, verbose)
            })
        self.area_results_df = pd.DataFrame(results_list)

        # area estimate is the fraction of sampled points in the feasible region
        # TODO: fix ;)
        self.area_estimate = self.area_results_df['in_feasible_region'].mean()

        return(self.area_estimate)

    def plot_both(self, axis=None, verbose=False, granularity=100, plot_segments=False):
      if axis is None:
          plt.style.use('seaborn-whitegrid')
          fig = plt.figure(figsize=(6, 4), dpi=150)
          axis = plt.axes()
          axis.set(
              xlim=(0, 1), ylim=(0, 1), xlabel='PPV', ylabel='FNR'
          )
      for i, (x_val, y_val, in_region) in tqdm(self.area_results_df.iterrows(), total=len(self.area_results_df), desc='Plotting sampled points', disable=not verbose):
          if in_region:
              axis.scatter(x_val, y_val, color='blue')
          else:
              axis.scatter(x_val, y_val, color='gray', alpha=0.5)
      num_blue_points = sum(self.area_results_df['in_feasible_region'])
      num_points = len(self.area_results_df['in_feasible_region'])
      axis.set_title(f'Area estimate: {num_blue_points}/{num_points} = {self.area_estimate}')

      x_grid = np.linspace(0, 1, granularity)
      for line in tqdm(self.lines, desc='Plotting lines', disable=not verbose):
          line.plot(x_vals=x_grid, axis=axis)

      if plot_segments:
          for segment in self.closest_segments:
              axis.plot(segment[0], segment[1], color='grey', marker='x', linestyle='dashed', scalex=False, scaley=False)

      if axis is None:
          plt.show()

In [ ]:
results_df_columns_2 = ['p1','p2','epsilon','ppv_window','area_estimate']
results_df_2 = pd.DataFrame()

base_rate_list = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]
#base_rate_step = 0.1
epsilon_values = [0.05] #[0.02, 0.05, 1.0]
epsilon_steps = 10
grid_points = 10
ppv_window = [0.5,1.0]
tol = ((1/grid_points) / 2)
list_of_ppv_windows = [[0,1.0]] #[[0,0.24],[0.25,0.49],[0.5,0.74],[0.75,0.99]]

for epsilon in epsilon_values:
    for ppv_window in list_of_ppv_windows:
        for base_rate_1 in base_rate_list:
            for base_rate_2 in base_rate_list:
                epsilon_range = np.arange((-1*epsilon), epsilon, (epsilon/(epsilon_steps/2)))

                print(f"DEBUG: Working, p1 = {base_rate_1}, p2 = {base_rate_2}, epsilon={epsilon}, ppv_window={ppv_window}, time=", end="")
                
                #line_group = GroupOfLines(base_rate_1, base_rate_2, epsilon_range, ppv_window=ppv_window, verbose=False)
                start = time.time()
                line_group = DetectorLattice(base_rate_1, base_rate_2, epsilon_range, ppv_window=ppv_window, verbose=False)
                line_group.compute_area(num_grid_points=grid_points, tolerance=tol, verbose=False)
                end = time.time()
                print(f'{end-start}s')
                
                r = {'p1': base_rate_1,
                    'p2': base_rate_2,
                    'epsilon': epsilon,
                    'ppv_window_low': ppv_window[0],
                    'ppv_window_high': ppv_window[1],
                    'area_estimate': line_group.area_estimate}
                results_df_2 = results_df_2.append(r, ignore_index=True)

time_str = str(time.localtime()[0])
for i in range(1,6):
    time_str = time_str + '_' + str(time.localtime()[i])

results_df_2.to_csv(f'results_df_2_{time_str}.csv')